
<h3>Querying the Recreational Information Database (RIDB)</h3>

<img src="images/legolas.jpg" width=30% style="display: inline-block">
<br>
image source: jadensadventures.wikia.com
<br>
A well documented, actively maintained API with a responsive development team

<br>
[RIDB homepage](https://ridb.recreation.gov/)
![](images/RIDB_header.png)

<br>
[Facilities Endpoint Documentation](http://usda.github.io/RIDB/#get-all-facilities)
![](images/ridb_api_detail.png)


In [5]:
facilities_url = 'https://ridb.recreation.gov/api/v1/facilities'

# camping activity ID is 9
activity_id = 9
limit = 50
offset = 0

In [51]:
import requests
import pandas as pd
import json
import math
from pandas.io.json import json_normalize
import config
import re

In [7]:
pd.options.display.max_columns

20

In [43]:
def get_facilities():

    facilities_params = dict(apiKey=config.RIDB_API_KEY, state='OR', activity=activity_id, limit=limit, offset=0)
    facilities_resp = requests.get(url=facilities_url, params=facilities_params)
    data = json.loads(facilities_resp.text)
    cg_info = pd.DataFrame()

    print("Got cg info")

    num_entries = data['METADATA']['RESULTS']['TOTAL_COUNT']
    print ("total: " + str(num_entries))
    num_iterations = int(math.ceil(float(num_entries)/float(limit)))
    print ("num iterations: " + str(num_iterations))

    for iter_i in range(1,num_iterations+1):
        print ("Number of json records: " + str(len(data['RECDATA'])))
        for i in data['RECDATA']:
            fac_name = i['FacilityName']
            fac_id = i['FacilityID']
            fac_lat = i['FacilityLatitude']
            fac_long = i['FacilityLongitude']
            #fac_resurl = i['FacilityReservationURL'] # not yet populated...
            cg_info = cg_info.append(pd.DataFrame({'Name' : [fac_name], 'Fac_Id' : [fac_id], 'Lat' : [fac_lat], \
                                                   'Long' : [fac_long]}))

        offset = 50*iter_i
        facilities_params = dict(apiKey= config.RIDB_API_KEY, state='OR', activity=activity_id, limit=limit, offset=offset)
        facilities_resp = requests.get(url=facilities_url, params=facilities_params)
        data = json.loads(facilities_resp.text)


    return cg_info 

In [44]:
ridb_data = get_facilities()

Got cg info
total: 811
num iterations: 17
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 50
Number of json records: 11


In [45]:
ridb_data.shape

(811, 4)

In [42]:
ridb_data.head()

,Fac_Id,Lat,Long,Name
0,210606,43.498,-123.494,Eagleview Group Campground
0,210607,44.7777,-119.624,Lone Pine Group Campground
0,224333,43.485,-123.484,Tyee Recreation Site
0,231800,42.9092,-122.057,Crater Lake Lodge
0,231978,45.2839,-123.96,SANDBEACH


In [65]:
ridb_data[ridb_data['Name'].apply(lambda x: re.search('wildwood', x.lower()) != None)]

,Fac_Id,Lat,Long,Name
0,234075,45.3561,-121.987,WILDWOOD RECREATION SITE
0,236929,44.4839,-120.337,Wildwood Campground


In [29]:
ridb_data['Fac_Resurl'].unique()

array(['', ' '], dtype=object)